<a href="https://colab.research.google.com/github/dohb128/BigData/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **데이터 수집**

In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json
import urllib.parse

client_id = 'pzhBfPcGK6NuBiQrGHuM'
client_secret = 'EN7jUsuFHZ'

def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL: %s" % (datetime.datetime.now(), url))
        return None

def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url)

    if responseDecode is None:
        return None
    else:
        return json.loads(responseDecode)

def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'cnt': cnt, 'title': title, 'description': description, 'org_link': org_link, 'link': link, 'pDate': pDate})
    return

def main():
    node = 'news'
    player_names = [
        '노시환', '홍창기', '오스틴', '최정', '서진용', '구자욱', '박영현', '손아섭', '알칸타라',
        '페디', '고영표', '뷰캐넌', '후라도', '김혜성', '쿠에바스', '정수빈'
    ]
    cnt = 0
    jsonResult = []

    for player in player_names:
        srcText = player + ' KBO MVP'  # 각 선수 이름과 'KBO MVP'를 조합
        start = 1  # 검색 결과의 시작 위치

        while True:
            jsonResponse = getNaverSearch(node, srcText, start, 100)

            if jsonResponse is None:
                print("검색 결과가 없거나 오류가 발생했습니다.")
                break

            total = jsonResponse['total']

            for post in jsonResponse['items']:
                pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
                # 2023년에 해당하는 결과만 가져옴
                if pDate.year == 2023 and 3 <= pDate.month <= 11:   # 23년 3월부터 11월까지의 기사 수집
                    cnt += 1
                    getPostData(post, jsonResult, cnt)

            start += 100  # 다음 페이지 검색 시작 위치

            if start > total:
                break  # 모든 검색 결과를 가져온 경우 종료

    print('전체 검색 : %d 건' % total)

    with open('KBO_MVP_후보_naver_%s.json' % node, 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys=True, ensure_ascii=False)
        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" % (cnt))
    print('KBO_MVP_후보_naver_%s.json SAVED' % node)

if __name__ == '__main__':
    main()

